Imports

In [ ]:
from enum import unique
from scipy.sparse.construct import rand
from sklearn.utils import shuffle
import medical_lib as ml
import pandas as pd
import numpy as np
import time
import warnings
warnings.filterwarnings("ignore")

Dataframe einlesen

In [ ]:
df = pd.read_csv('filled_noStr.csv')
df = df.iloc[:, 1:]
print(df.head())

tot = pd.read_csv('Verstorben.csv')

'Filled_noStr.csv' wird in zwei Datasets geteilt.
Df1 enthält nur die Toten. Df2 enthält nur die Lebenden.
Anschliessend werden die Daten von 94 Toten und 94 Lebenden zusammengeführt und die Daten der restlichen 20 Toten und restlichen Lebenden separat vereinigt.

Diese beiden Dataframes werden später Train und Testset sein.

In [ ]:
uniquetote = tot['Pseudonym'].unique().tolist()
df1 = df[df['Pseudonym'].isin(uniquetote)] #Dataframe nur mit den Toten
df2 = df[~df['Pseudonym'].isin(uniquetote)] #Dataframe nur mit den Lebenden
uniquelebende = df2['Pseudonym'].unique().tolist() #Liste aller distinct Lebende

uniquetote1 = uniquetote[0:(len(uniquetote)-20)] #Liste aus 94 tote
uniquetote2 = uniquetote[len(uniquetote)-20: len(uniquetote)] #Liste der restlichen 20tote

uniquelebende1 = uniquelebende[0: 94] #Liste aus 94Lebende
uniquelebende2 = uniquelebende[94:len(uniquelebende)] # Liste der restlichen Lebenden

beides_Train = uniquetote1 + uniquelebende1
beides_Test = uniquetote1 + uniquelebende2

In [ ]:
df1v3_train = df[df['Pseudonym'].isin(beides_Train)] # Dataframe aus 94tote und 94lebende. Komplettes Train-Set
df1v3_Test = df[df['Pseudonym'].isin(beides_Test)]

In [ ]:
df1v3_train = df[df['Pseudonym'].isin(beides_Train)] # Dataframe aus 94tote und 94lebende. Komplettes Train-Set
df1v3_Test = df[df['Pseudonym'].isin(beides_Test)]
df1v3_train.to_csv('model1v3trainunfilled.csv')
df1v3_Test.to_csv('model1v3testunfilled.csv')

Fülle model1v3trainunfilled mit den Werten direkt nach OP, d.h. entweder an Tag 0, falls Wert vorhanden, oder unmittelbar danach.

In [ ]:
df = pd.read_csv('model1v3trainunfilled.csv')
df = df.iloc[:, 1:]
print(df.head())

Für jeden Patienten: Nehme einen Ausschnitt von Tag 0 bis 100 aus dem Datensatz als neues Dataframe. Gehe durch alle Zeilen 

In [34]:
def takefreshop(inputDF, pseudo):
    dffilled = inputDF.copy()
    p = dffilled[dffilled['Pseudonym'] == pseudo]
    p.sort_values(by='relatives_datum', ascending=True, inplace=True)
    col = dffilled.columns
    # tmp.to_csv('0.csv')
    naive = pd.DataFrame(columns=col)
    abkuset = set()
    for row in range(0, len(p)):
        for col in range(0, len(p.columns)):
            value = p.iloc[row, col]
            if (np.isnan(value) == False) and (abkuset.__contains__(p.columns[col]) == False): 
                naive.loc[0, p.columns[col]] = value
                abkuset.add(p.columns[col])
                continue
                
    return naive

In [35]:
#Erstelle Dataframe, in der für jeden Patienten nur sein aktuellster Wert pro Feature benutzt wird
inputDf = 'model1v3trainunfilled.csv'
outputDf = 'model1v3train_Filled.csv'
        

df = pd.read_csv(inputDf)
df = df.iloc[:, 1:]
op = df[df['relatives_datum'].between(0,100)]
pseudolist = df['Pseudonym'].unique()
frames = []
i = 0
for name in pseudolist:
    tmp = takefreshop(op, name)
    frames.append(tmp)
    i += 1
result = pd.concat(frames)
# result.to_csv(outputDf)

In [36]:
print(result.head())

  Pseudonym relatives_datum    CA    CREA   CRP   ERY   EVB  GGT37 GOT37  \
0       0.0             0.0  1.99  0.6801  2.61  3.54  13.7  284.0  86.0   
0      79.0             0.0  1.89  0.5501  0.61  2.67  12.8   38.0   7.0   
0     124.0             0.0  2.11  0.4101  1.01  2.77  18.7  112.0  10.0   
0     247.0             0.0  2.26  0.5501  1.66  2.95  24.2  104.0  88.0   
0     997.0             0.0  2.45  0.8501  0.97  3.32  15.9  167.0  15.0   

   GPT37  ...   IL-6 G.APTT G.QUICK   IRF CA-KORR NRBC-ABS FK-RO IG-ABS  \
0  254.0  ...    NaN    NaN     NaN   NaN     NaN      NaN   NaN    NaN   
0   20.0  ...   8.72   22.9    71.7   1.2    2.24     0.03  5.08    0.1   
0   20.0  ...   99.6   26.1    77.0   0.0  2.3325     0.03   NaN   0.11   
0   33.0  ...  54.47   31.3    80.6  21.2  2.3275     0.05   1.0    NaN   
0   34.0  ...   5.29   29.3    85.9  53.9  2.4325     0.03  9.94   0.13   

  IG-REL   CSA-RO  
0    NaN      NaN  
0    2.8      NaN  
0    1.7      NaN  
0    NaN  30